# Advanced Data Wrangling & Analysis

## Lesson Overview

This workshop transforms basic Python knowledge into professional data manipulation skills. We follow the "Extract-Transform-Analyze" workflow across 4 distinct sections:

1.  **Part 1: Financial Time Series & Window Functions**
    * Handling Datetime objects and indexing
    * Resampling and Frequency conversion
    * Window functions (Rolling means)
    * Covariance and Correlation
2.  **Part 2: Data Wrangling (Merge & Reshape)**
    * Merging datasets (Inner, Outer, Left, Right joins)
    * Reshaping data: Melt and Pivot
3.  **Part 3: Aggregation & Reporting**
    * GroupBy mechanics (Split-Apply-Combine)
    * Pivot Tables and Cross-Tabulations
4.  **Part 4: Advanced Toolkit (Optional/Deep Dive)**
    * Hierarchical Indexing (MultiIndex)
    * Concatenation
    * Stacking/Unstacking
    * Advanced GroupBy: Apply and Transform

---

**Setup:** Import necessary libraries.

In [4]:
import pandas as pd
import numpy as np
from datetime import datetime

# Part 1: Financial Time Series & Window Functions

**Learning Objective:** Handle data where the *order* matters (Time Series). We will learn to convert strings to dates, handle missing business days, smoothing volatile data, and analyzing stock correlations.

## 1.1 Handling Date Time Data

Pandas is oriented towards working with arrays of dates, whether used as an axis index or a column.

The `to_datetime` method parses may different kinds of date representations:

In [5]:
dates = ["2011-07-06 12:00:00", "2011-08-06 00:00:00"]

pd.to_datetime(dates)

DatetimeIndex(['2011-07-06 12:00:00', '2011-08-06 00:00:00'], dtype='datetime64[ns]', freq=None)

It uses `NaT` (Not a Time) as null values for datetime data.

In [6]:
idx = pd.to_datetime(dates + [None])
idx

DatetimeIndex(['2011-07-06 12:00:00', '2011-08-06 00:00:00', 'NaT'], dtype='datetime64[ns]', freq=None)

In [4]:
pd.isna(idx)

array([False, False,  True])

Standard Python uses the `datetime` module to handle date and time data. Pandas has a `Timestamp` object that is similar to the `datetime` object. 

If you use `datetime` objects as index to a Series or DataFrame, Pandas will automatically convert them to `DatetimeIndex` objects.

In [7]:
dates = [datetime(2011, 1, 2), datetime(2011, 1, 5), datetime(2011, 1, 7), 
         datetime(2011, 1, 8), datetime(2011, 1, 10), datetime(2011, 1, 12)]

ts = pd.Series(np.random.standard_normal(6), index=dates)
ts

2011-01-02    1.183094
2011-01-05   -1.855339
2011-01-07    2.393560
2011-01-08    0.428285
2011-01-10   -0.978827
2011-01-12   -0.618416
dtype: float64

In [8]:
ts.index

DatetimeIndex(['2011-01-02', '2011-01-05', '2011-01-07', '2011-01-08',
               '2011-01-10', '2011-01-12'],
              dtype='datetime64[ns]', freq=None)

Like other Series, arithmetic operations between differently indexed time series automatically align on the dates:

In [9]:
# [::2] selects every second element
ts + ts[::2]

2011-01-02    2.366188
2011-01-05         NaN
2011-01-07    4.787119
2011-01-08         NaN
2011-01-10   -1.957655
2011-01-12         NaN
dtype: float64

In [10]:
ts[::2]

2011-01-02    1.183094
2011-01-07    2.393560
2011-01-10   -0.978827
dtype: float64

### Indexing & Slicing

You can index by passing a `datetime`, `Timestamp` or `string` that is interpretable as a date:

In [11]:
ts[datetime(2011, 1, 7)]

2.393559592751022

In [10]:
ts[pd.Timestamp("2011-01-07")]

1.9661074891045567

In [12]:
ts["2011-01-07"]

2.393559592751022

You can even specify the year or year-month strings to slice a range of data. This is very powerful for quick analysis.

In [14]:
# date_range generate an array of dates
longer_ts = pd.Series(np.random.standard_normal(1000), 
                      index=pd.date_range("2000-01-01", periods=1000))

longer_ts

2000-01-01    0.357761
2000-01-02   -0.337404
2000-01-03    0.688203
2000-01-04   -1.358423
2000-01-05   -0.028225
                ...   
2002-09-22   -0.963562
2002-09-23    0.058814
2002-09-24    0.948851
2002-09-25    0.450972
2002-09-26    0.776396
Freq: D, Length: 1000, dtype: float64

In [15]:
pd.date_range("2000-01-01", periods=1000, freq="10H")

DatetimeIndex(['2000-01-01 00:00:00', '2000-01-01 10:00:00',
               '2000-01-01 20:00:00', '2000-01-02 06:00:00',
               '2000-01-02 16:00:00', '2000-01-03 02:00:00',
               '2000-01-03 12:00:00', '2000-01-03 22:00:00',
               '2000-01-04 08:00:00', '2000-01-04 18:00:00',
               ...
               '2001-02-16 12:00:00', '2001-02-16 22:00:00',
               '2001-02-17 08:00:00', '2001-02-17 18:00:00',
               '2001-02-18 04:00:00', '2001-02-18 14:00:00',
               '2001-02-19 00:00:00', '2001-02-19 10:00:00',
               '2001-02-19 20:00:00', '2001-02-20 06:00:00'],
              dtype='datetime64[ns]', length=1000, freq='10H')

In [16]:
# Select all data from 2001
longer_ts["2001"].head()

2001-01-01   -0.748196
2001-01-02    0.150133
2001-01-03   -0.556804
2001-01-04    0.849377
2001-01-05   -0.335949
Freq: D, dtype: float64

In [17]:
# Select all data from May 2001
longer_ts["2001-05"].head()

2001-05-01    1.630507
2001-05-02   -0.860589
2001-05-03   -0.802391
2001-05-04   -0.677114
2001-05-05    2.494096
Freq: D, dtype: float64

In [18]:
# Slicing with strings
longer_ts["2001-05":].head()

2001-05-01    1.630507
2001-05-02   -0.860589
2001-05-03   -0.802391
2001-05-04   -0.677114
2001-05-05    2.494096
Freq: D, dtype: float64

**Student Exercise:** Use `date_range` to generate a Series of random values from 1-31st January 2023. Then slice the Series to return data from 5-15th January.

In [19]:
data_range = pd.Series(np.random.standard_normal(31), 
                       index=pd.date_range("2023-01-01", periods=31))

data_range["2023-01-05":"2023-01-15"]

2023-01-05   -0.801180
2023-01-06    1.462265
2023-01-07   -0.269867
2023-01-08    0.191554
2023-01-09   -0.197770
2023-01-10   -0.291927
2023-01-11    0.667542
2023-01-12    1.131448
2023-01-13   -0.422982
2023-01-14    1.193832
2023-01-15    0.481947
Freq: D, dtype: float64

## 1.2 Real-World Application: Stock Market Data

Let's load stock prices (AAPL, GOOG, IBM, MSFT) and trade volumes. We use `.read_pickle` here as it preserves the datetime objects native to the file.

In [20]:
price = pd.read_pickle("../data/yahoo_price.pkl")
volume = pd.read_pickle("../data/yahoo_volume.pkl")

In [20]:
price.head()

,AAPL,GOOG,IBM,MSFT
Date,,,,
2010-01-04,27.990226,313.062468,113.304536,25.884104
2010-01-05,28.038618,311.683844,111.935822,25.892466
2010-01-06,27.592626,303.826685,111.208683,25.733566
2010-01-07,27.541619,296.753749,110.823732,25.465944
2010-01-08,27.724725,300.709808,111.935822,25.641571


In [21]:
volume.head()

,AAPL,GOOG,IBM,MSFT
Date,,,,
2010-01-04,123432400,3927000,6155300,38409100
2010-01-05,150476200,6031900,6841400,49749600
2010-01-06,138040000,7987100,5605300,58182400
2010-01-07,119282800,12876600,5840600,50559700
2010-01-08,111902700,9483900,4197200,51197400


### Inspecting the Index
Notice the index is a `DatetimeIndex`.

In [22]:
price.index

DatetimeIndex(['2010-01-04', '2010-01-05', '2010-01-06', '2010-01-07',
               '2010-01-08', '2010-01-11', '2010-01-12', '2010-01-13',
               '2010-01-14', '2010-01-15',
               ...
               '2016-10-10', '2016-10-11', '2016-10-12', '2016-10-13',
               '2016-10-14', '2016-10-17', '2016-10-18', '2016-10-19',
               '2016-10-20', '2016-10-21'],
              dtype='datetime64[ns]', name='Date', length=1714, freq=None)

We can access attributes like `day_of_week` directly:

In [23]:
price.index.day_of_week

Int64Index([0, 1, 2, 3, 4, 0, 1, 2, 3, 4,
            ...
            0, 1, 2, 3, 4, 0, 1, 2, 3, 4],
           dtype='int64', name='Date', length=1714)

In [24]:
price.index.month

Int64Index([ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
            ...
            10, 10, 10, 10, 10, 10, 10, 10, 10, 10],
           dtype='int64', name='Date', length=1714)

If the datetime is in a column instead of the index, you can use the `dt` accessor to access the datetime properties.

In [25]:
price_reindex = price.reset_index()
price_reindex.head()

,Date,AAPL,GOOG,IBM,MSFT
0,2010-01-04,27.990226,313.062468,113.304536,25.884104
1,2010-01-05,28.038618,311.683844,111.935822,25.892466
2,2010-01-06,27.592626,303.826685,111.208683,25.733566
3,2010-01-07,27.541619,296.753749,110.823732,25.465944
4,2010-01-08,27.724725,300.709808,111.935822,25.641571


In [26]:
price_reindex["Date"].dt.day_name().head()

0       Monday
1      Tuesday
2    Wednesday
3     Thursday
4       Friday
Name: Date, dtype: object

**Student Exercise:** Get the week of year from the date column and create a new column `week_of_year`.

In [27]:
price_reindex["week_of_year"] = price_reindex["Date"].dt.isocalendar().week
price_reindex.head()

,Date,AAPL,GOOG,IBM,MSFT,week_of_year
0,2010-01-04,27.990226,313.062468,113.304536,25.884104,1
1,2010-01-05,28.038618,311.683844,111.935822,25.892466,1
2,2010-01-06,27.592626,303.826685,111.208683,25.733566,1
3,2010-01-07,27.541619,296.753749,110.823732,25.465944,1
4,2010-01-08,27.724725,300.709808,111.935822,25.641571,1


## 1.3 Resampling (Frequency Conversion)

As you can see from above, the dates are on business days. If you want to change the frequency to calendar days (e.g. including weekends), we use `resample`.

This introduces missing data (NaN) for weekends.

In [32]:
price_resampled = price.resample('D').asfreq()
price_resampled.head(10)

,AAPL,GOOG,IBM,MSFT
Date,,,,
2010-01-04,27.990226,313.062468,113.304536,25.884104
2010-01-05,28.038618,311.683844,111.935822,25.892466
2010-01-06,27.592626,303.826685,111.208683,25.733566
2010-01-07,27.541619,296.753749,110.823732,25.465944
2010-01-08,27.724725,300.709808,111.935822,25.641571
2010-01-09,NaN,NaN,NaN,NaN
2010-01-10,NaN,NaN,NaN,NaN
2010-01-11,27.480148,300.255255,110.763844,25.315406
2010-01-12,27.167562,294.945572,111.644958,25.148142


**Forward Fill (`ffill`)**: If you want to fill the na values with the most recent value (common in finance - Saturday price is same as Friday close), you can use the `.ffill()` method.

In [33]:
price_resampled = price.resample('D').ffill()
price_resampled.head(10)

,AAPL,GOOG,IBM,MSFT
Date,,,,
2010-01-04,27.990226,313.062468,113.304536,25.884104
2010-01-05,28.038618,311.683844,111.935822,25.892466
2010-01-06,27.592626,303.826685,111.208683,25.733566
2010-01-07,27.541619,296.753749,110.823732,25.465944
2010-01-08,27.724725,300.709808,111.935822,25.641571
2010-01-09,27.724725,300.709808,111.935822,25.641571
2010-01-10,27.724725,300.709808,111.935822,25.641571
2010-01-11,27.480148,300.255255,110.763844,25.315406
2010-01-12,27.167562,294.945572,111.644958,25.148142


If you want to resample to a **lower frequency** (e.g. Monthly 'MS' - Month Start) you need to provide an aggregation method (like `mean`):

In [34]:
price_resampled = price.resample('MS').mean()
price_resampled.head(20)

,AAPL,GOOG,IBM,MSFT
Date,,,,
2010-01-01,27.166942,289.012006,110.364037,25.212407
2010-02-01,26.000370,267.179857,107.496919,23.767984
2010-03-01,29.219760,280.235245,109.849295,24.584058
2010-04-01,32.847556,278.253415,111.324726,25.646245
2010-05-01,32.888483,248.418509,109.690449,23.670255
2010-06-01,34.155041,241.994331,110.141435,21.455428
2010-07-01,33.344742,236.247254,110.172506,21.163254
2010-08-01,32.870393,239.417581,111.032612,20.907435
2010-09-01,35.816785,247.020786,112.992065,20.869261


**Student Exercise:** Resample price to `yearly` (start of year) frequency, use `sum` as aggregation function.

In [35]:
price.resample('YS').sum().head()

,AAPL,GOOG,IBM,MSFT
Date,,,,
2010-01-01,8564.125188,67421.204377,28722.464346,5757.023922
2011-01-01,11997.187091,71619.248624,37895.630612,5675.814809
2012-01-01,18880.993859,80271.925770,43947.570175,6621.497457
2013-01-01,15929.515474,111303.472578,44524.331233,7498.810042
2014-01-01,22293.854902,141458.282752,42727.132433,10081.875149


## 1.4 Window Functions (Moving Averages)

You can apply functions evaluated over a sliding window using the `rolling` method.

For example, to compute the 30-day moving average for Apple price:

In [36]:
price["AAPL"].rolling(30).mean().tail()

Date
2016-10-17    112.730333
2016-10-18    113.056000
2016-10-19    113.348000
2016-10-20    113.732666
2016-10-21    114.181667
Name: AAPL, dtype: float64

By default, rolling functions require all of the values in the window to be non-NA. This behavior can be changed to account for missing data and, especially at the beginning of the time series.

In [37]:
price["AAPL"].rolling(30, min_periods=3).mean().head()

Date
2010-01-04          NaN
2010-01-05          NaN
2010-01-06    27.873823
2010-01-07    27.790772
2010-01-08    27.777563
Name: AAPL, dtype: float64

**Student Exercise:** Compute a 10-day moving average for `GOOG` with a min period of 5 days.

In [38]:
price["GOOG"].rolling(10, min_periods=5).mean().tail()

Date
2016-10-17    779.667004
2016-10-18    781.550006
2016-10-19    784.053009
2016-10-20    786.064007
2016-10-21    788.493005
Name: GOOG, dtype: float64

## 1.5 Covariance and Correlation

Covariance and correlation measure the relationship between two variables.

* **Covariance:** Measure of how much two random variables vary together. Hard to interpret magnitude.
* **Correlation:** Normalized measure (-1 to 1). 1 is perfect positive correlation, -1 is perfect negative.

In finance, we usually look at **Returns** (Percent Change), not raw prices.

In [39]:
returns = price.pct_change()
returns.tail()

,AAPL,GOOG,IBM,MSFT
Date,,,,
2016-10-17,-0.000680,0.001837,0.002072,-0.003483
2016-10-18,-0.000681,0.019616,-0.026168,0.007690
2016-10-19,-0.002979,0.007846,0.003583,-0.002255
2016-10-20,-0.000512,-0.005652,0.001719,-0.004867
2016-10-21,-0.003930,0.003011,-0.012474,0.042096


Compute the correlation and covariance between the returns of `MSFT` and `IBM`:

In [40]:
print("Covariance:", returns["MSFT"].cov(returns["IBM"]))
print("Correlation:", returns["MSFT"].corr(returns["IBM"]))

Covariance: 8.870655479703546e-05
Correlation: 0.4997636114415114


You can also get the full (pair-wise) correlation or covariance matrix as a DataFrame:

In [41]:
returns.corr()

,AAPL,GOOG,IBM,MSFT
AAPL,1.000000,0.407919,0.386817,0.389695
GOOG,0.407919,1.000000,0.405099,0.465919
IBM,0.386817,0.405099,1.000000,0.499764
MSFT,0.389695,0.465919,0.499764,1.000000


You can also compute pair-wise correlations between a DataFrame’s columns or rows with another Series or DataFrame.

In [42]:
# Correlation of all companies against IBM
returns.corrwith(returns["IBM"])

AAPL    0.386817
GOOG    0.405099
IBM     1.000000
MSFT    0.499764
dtype: float64

In [43]:
# Correlation of returns against volume
returns.corrwith(volume)

AAPL   -0.075565
GOOG   -0.007067
IBM    -0.204849
MSFT   -0.092950
dtype: float64

---
# Part 2: Data Wrangling (Merge & Reshape)

**Learning Objective:** Combine data from different sources (SQL-style Joins) and reshape table layouts (Wide to Long) to prepare for analysis.

## 2.1 Merging (Joins)

`merge` connects rows in DataFrames based on one or more keys. This is equivalent to database `join` operations.

In [44]:
df1 = pd.DataFrame({"key": ["b", "b", "a", "c", "a", "a", "b"], 
                    "data1": pd.Series(range(7), dtype="Int64")})

df2 = pd.DataFrame({"key": ["a", "b", "d"], 
                    "data2": pd.Series(range(3), dtype="Int64")})

print("DF1 (Left):\n", df1)
print("\nDF2 (Right):\n", df2)

DF1 (Left):
   key  data1
0   b      0
1   b      1
2   a      2
3   c      3
4   a      4
5   a      5
6   b      6

DF2 (Right):
   key  data2
0   a      0
1   b      1
2   d      2


**Many-to-One Join:** `df1` has multiple rows labeled `a` and `b`, whereas `df2` has only one row for each value in the key column `key`.

The default is an **Inner Join** (intersection of keys).

In [45]:
pd.merge(df1, df2)

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


It is good practice to specify the key explicitly:

In [46]:
pd.merge(df1, df2, on="key")

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


If the column names are different in each object, you can specify them separately using `left_on` and `right_on`:

In [47]:
df3 = pd.DataFrame({"lkey": ["b", "b", "a", "c", "a", "a", "b"], 
                    "data1": pd.Series(range(7), dtype="Int64")})

df4 = pd.DataFrame({"rkey": ["a", "b", "d"], 
                    "data2": pd.Series(range(3), dtype="Int64")})

pd.merge(df3, df4, left_on="lkey", right_on="rkey")

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


### Join Types (Inner, Outer, Left, Right)

You can specify the other options via the `how` parameter.

In [48]:
# Outer Join: Union of keys. Fills missing with NaN
pd.merge(df1, df2, how="outer")

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0
6,c,3,<NA>
7,d,<NA>,2


In [49]:
# Outer Join with mismatched key names
pd.merge(df3, df4, left_on="lkey", right_on="rkey", how="outer")

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0
6,c,3,NaN,<NA>
7,NaN,<NA>,d,2


**Many-to-Many Join:**

In [50]:
df1 = pd.DataFrame({"key": ["b", "b", "a", "c", "a", "b"], 
                    "data1": pd.Series(range(6), dtype="Int64")})

df2 = pd.DataFrame({"key": ["a", "b", "a", "b", "d"], 
                    "data2": pd.Series(range(5), dtype="Int64")})

pd.merge(df1, df2, how="inner")

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,b,5,1
5,b,5,3
6,a,2,0
7,a,2,2
8,a,4,0
9,a,4,2


**Student Exercise:** Merge `df1` and `df2` with a left join.

In [51]:
pd.merge(df1, df2, how="left")

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,a,2,0
5,a,2,2
6,c,3,<NA>
7,a,4,0
8,a,4,2
9,b,5,1


### Merging on Multiple Keys & Suffixes

To merge with multiple keys, pass a list of column names:

In [52]:
left = pd.DataFrame({"key1": ["foo", "foo", "bar"], 
                     "key2": ["one", "two", "one"],
                     "lval": pd.Series([1, 2, 3], dtype='Int64')})

right = pd.DataFrame({"key1": ["foo", "foo", "bar", "bar"],
                      "key2": ["one", "one", "one", "two"],
                      "rval": pd.Series([4, 5, 6, 7], dtype='Int64')})

pd.merge(left, right, on=["key1", "key2"], how="outer")

,key1,key2,lval,rval
0,foo,one,1,4
1,foo,one,1,5
2,foo,two,2,<NA>
3,bar,one,3,6
4,bar,two,<NA>,7


If there are overlapping non-key column names, `merge` adds suffixes `_x` and `_y` by default. You can customize this:

In [53]:
pd.merge(left, right, on="key1")

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [54]:
pd.merge(left, right, on="key1", suffixes=("_left", "_right"))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


### Merging on Index

If the merge key(s) is in the index, you can pass `left_index=True` or `right_index=True`.

In [55]:
left1 = pd.DataFrame({"key": ["a", "b", "a", "a", "b", "c"],
                      "value": pd.Series(range(6), dtype="Int64")})

right1 = pd.DataFrame({"group_val": [3.5, 7]}, index=["a", "b"])

pd.merge(left1, right1, left_on="key", right_index=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


DataFrame has a `join` method which performs a left join by default. It's a convenient shortcut for index-on-index merging.

In [57]:
left1.join(right1, on='key')

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


## 2.2 Reshaping and Pivoting

We often need to switch between **Wide Format** (Excel style, years as columns) and **Long Format** (Database style, one row per observation).

### Melt (Wide to Long)
Let's look at our stock price data. It is currently **Wide**.

In [58]:
# Reset index so Date is a column
price_reindex = price.reset_index()
price_reindex.head()

,Date,AAPL,GOOG,IBM,MSFT
0,2010-01-04,27.990226,313.062468,113.304536,25.884104
1,2010-01-05,28.038618,311.683844,111.935822,25.892466
2,2010-01-06,27.592626,303.826685,111.208683,25.733566
3,2010-01-07,27.541619,296.753749,110.823732,25.465944
4,2010-01-08,27.724725,300.709808,111.935822,25.641571


In [70]:
# Melt into Long format
melted = pd.melt(price_reindex, id_vars="Date")
melted.head()

,Date,variable,value
0,2010-01-04,AAPL,27.990226
1,2010-01-05,AAPL,28.038618
2,2010-01-06,AAPL,27.592626
3,2010-01-07,AAPL,27.541619
4,2010-01-08,AAPL,27.724725


**Student Exercise:** Rerun `melt` and pass arguments such that the new columns are named `Company` and `Price` respectively.

In [71]:
melted = pd.melt(price_reindex, id_vars="Date", var_name="Company", value_name="Price")
melted.head()

,Date,Company,Price
0,2010-01-04,AAPL,27.990226
1,2010-01-05,AAPL,28.038618
2,2010-01-06,AAPL,27.592626
3,2010-01-07,AAPL,27.541619
4,2010-01-08,AAPL,27.724725


### Pivot (Long to Wide)
Using `pivot`, we can reshape back to the original layout:

In [96]:
melted

,Date,Company,Price
0,2010-01-04,AAPL,27.990226
1,2010-01-05,AAPL,28.038618
2,2010-01-06,AAPL,27.592626
3,2010-01-07,AAPL,27.541619
4,2010-01-08,AAPL,27.724725
...,...,...,...
6851,2016-10-17,MSFT,57.220001
6852,2016-10-18,MSFT,57.660000
6853,2016-10-19,MSFT,57.529999
6854,2016-10-20,MSFT,57.250000


In [98]:
reshaped = melted.pivot(index='Date', columns='Company', values='Price')
reshaped.head()

Company,AAPL,GOOG,IBM,MSFT
Date,,,,
2010-01-04,27.990226,313.062468,113.304536,25.884104
2010-01-05,28.038618,311.683844,111.935822,25.892466
2010-01-06,27.592626,303.826685,111.208683,25.733566
2010-01-07,27.541619,296.753749,110.823732,25.465944
2010-01-08,27.724725,300.709808,111.935822,25.641571


---
# Part 3: Aggregation & Reporting

**Learning Objective:** Summarize data using GroupBy, Custom Aggregations, and Pivot Tables to answer business questions.

## 3.1 Data Aggregation (GroupBy)

Data aggregation is the process of grouping data together and performing calculations on them.

In [80]:
df = pd.DataFrame({"key1" : ["a", "a", None, "b", "b", "a", None], 
                   "key2" : pd.Series([1, 2, 1, 2, 1, None, 1], dtype="Int64"),
                   "data1" : np.random.standard_normal(7), 
                   "data2" : np.random.standard_normal(7)})
df

,key1,key2,data1,data2
0,a,1,0.844087,0.930259
1,a,2,1.467677,-1.252092
2,None,1,0.922059,0.905024
3,b,2,-0.120731,0.342627
4,b,1,-0.000324,0.747548
5,a,<NA>,-1.660105,-0.963586
6,None,1,-0.990969,1.069673


If you want to compute the mean for each unique value in `key1`:

In [81]:
df.groupby("key1").mean()

,key2,data1,data2
key1,,,
a,1.5,0.217219,-0.428473
b,1.5,-0.060528,0.545088


It does not make sense to compute the mean for `key2` since it is a categorical variable and also serves as a key.

We can select the numeric columns to compute the mean for (after the `groupby` method):

In [82]:
df.groupby("key1")[["data1", "data2"]].mean()

,data1,data2
key1,,
a,0.217219,-0.428473
b,-0.060528,0.545088


Note that the following also works, since the returned result is a DataFrame, however it is less efficient as the selection/subset happens after the computation.

In [83]:
df.groupby("key1").mean()[["data1", "data2"]]

,data1,data2
key1,,
a,0.217219,-0.428473
b,-0.060528,0.545088


You can group by more than 1 column. There is a useful GroupBy method `size` which returns a Series containing group sizes.

In [84]:
df.groupby(['key1', 'key2']).size()

key1  key2
a     1       1
      2       1
b     1       1
      2       1
dtype: int64

You can also group by other `Series`/`array`/`list` with the same length:

In [85]:
states = np.array(["OH", "CA", "CA", "OH", "OH", "CA", "OH"])
years = [2005, 2005, 2006, 2005, 2006, 2005, 2006]

df["data1"].groupby([states, years]).mean()

CA  2005   -0.096214
    2006    0.922059
OH  2005    0.361678
    2006   -0.495647
Name: data1, dtype: float64

**Student Exercise:** Group by `key1` and `key2` and compute the standard deviation.

## 3.2 Custom Aggregation

To use your own aggregation functions, pass any function that aggregates an array to the `aggregate` method or its short alias `agg`:

In [86]:
df

,key1,key2,data1,data2
0,a,1,0.844087,0.930259
1,a,2,1.467677,-1.252092
2,None,1,0.922059,0.905024
3,b,2,-0.120731,0.342627
4,b,1,-0.000324,0.747548
5,a,<NA>,-1.660105,-0.963586
6,None,1,-0.990969,1.069673


In [87]:
def peak_to_peak(arr):
    return arr.max() - arr.min()

In [88]:
grouped = df.groupby("key1")
grouped.agg(peak_to_peak)

,key2,data1,data2
key1,,,
a,1,3.127782,2.182350
b,1,0.120406,0.404921


You can pass a list of functions, or function names (for built-in functions) to `aggregate`: 

In [89]:
grouped.agg([peak_to_peak, "mean", "std"])

key2                       data1                      \
     peak_to_peak mean       std peak_to_peak      mean       std   
key1                                                                
a               1  1.5  0.707107     3.127782  0.217219  1.655439   
b               1  1.5  0.707107     0.120406 -0.060528  0.085140   

            data2                      
     peak_to_peak      mean       std  
key1                                   
a        2.182350 -0.428473  1.185505  
b        0.404921  0.545088  0.286323

## 3.3 Pivot Tables

Pivot tables are used to summarize, sort, reorganize, group, count, total or average data. It allows its users to transform columns into rows and rows into columns.

We will use the `tips.csv` dataset.

In [90]:
tips = pd.read_csv("../data/tips.csv")

# add a column with the tip percentage
tips["tip_pct"] = tips["tip"] / tips["total_bill"]

tips.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


The default aggregation for `pivot_table` is mean.

In [91]:
tips.pivot_table(index=["day", "smoker"], values=["size", "tip", "tip_pct", "total_bill"])

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

You can put `smoker` in the table columns and `time` and `day` in the rows:

In [92]:
tips.pivot_table(index=["time", "day"], columns="smoker", 
                 values=["tip_pct", "size"])

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

Add partial totals by passing `margins=True`:

In [99]:
tips.pivot_table(index=["time", "day"], columns="smoker", 
                 values=["tip_pct", "size"], margins=True)

size                       tip_pct                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.139622  0.165347  0.158916
       Sat   2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
       Sun   2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
       Thur  2.000000       NaN  2.000000  0.159744       NaN  0.159744
Lunch  Fri   3.000000  1.833333  2.000000  0.187735  0.188937  0.188765
       Thur  2.500000  2.352941  2.459016  0.160311  0.163863  0.161301
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

To use other aggregation functions, pass it to the `aggfunc` keyword:

In [100]:
tips.pivot_table(index=["time", "smoker"], columns="day", 
                 values="tip_pct", aggfunc=len, margins=True)

day             Fri   Sat   Sun  Thur  All
time   smoker                             
Dinner No       3.0  45.0  57.0   1.0  106
       Yes      9.0  42.0  19.0   NaN   70
Lunch  No       1.0   NaN   NaN  44.0   45
       Yes      6.0   NaN   NaN  17.0   23
All            19.0  87.0  76.0  62.0  244

Use `fill_value` to fill missing values:

In [101]:
tips.pivot_table(index=["time", "smoker"], columns="day", 
                 values="tip_pct", aggfunc=len, margins=True, fill_value=0)

day            Fri  Sat  Sun  Thur  All
time   smoker                          
Dinner No        3   45   57     1  106
       Yes       9   42   19     0   70
Lunch  No        1    0    0    44   45
       Yes       6    0    0    17   23
All             19   87   76    62  244

**Student Exercise:** Compute the sum of `tip` in a pivot table with `day` and `time` in the rows and `smoker` in the column.

In [102]:
tips.pivot_table(index=["day", "time"], columns="smoker", values="tip", aggfunc="sum")

smoker           No     Yes
day  time                  
Fri  Dinner    8.25   27.03
     Lunch     3.00   13.68
Sat  Dinner  139.63  120.77
Sun  Dinner  180.57   66.82
Thur Dinner    3.00     NaN
     Lunch   117.32   51.51

### Cross-Tabulation

A _cross-tabulation_ or _crosstab_ is a special case of pivot table that computes group frequencies (counts):

In [103]:
pd.crosstab(index=[tips["time"], tips["day"]], columns=tips["smoker"], margins=True)

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244

---
# Part 4: Advanced Toolkit (Optional / Deep Dive)

**Learning Objective:** Master complex data structures and advanced transformations. This section covers Hierarchical Indexing, Stacking, Concatenation, and custom Apply/Transform logic.

## 4.1 Hierarchical Indexing (MultiIndex)

Hierarchical indexing (MultiIndex) allows you to have multiple (two or more) _index levels_ on an axis. It enables "higher dimensional" data in a lower dimensional data structure.

In [104]:
data = pd.Series(np.random.uniform(size=9),
                 index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
                 [1, 2, 3, 1, 3, 1, 2, 2, 3]])
data

a  1    0.305092
   2    0.051239
   3    0.394671
b  1    0.370459
   3    0.214656
c  1    0.230757
   2    0.278646
d  2    0.719206
   3    0.103738
dtype: float64

In [ ]:
data.index

You can use _partial indexing_ to select subsets of data:

In [ ]:
data["b"]

In [ ]:
data["b":"c"]

In [ ]:
data.loc[["b", "d"]]

You can also select from "inner" level:

In [ ]:
data.loc[:, 2]

Hierarchical indexing works on both axes.

In [ ]:
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
                        index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                        columns=[['Ohio', 'Ohio', 'Colorado'],
                        ['Green', 'Red', 'Green']])
frame

Setting names on the axes work as usual:

In [ ]:
frame.index.names = ["key1", "key2"]
frame.columns.names = ["state", "color"]
frame

In [ ]:
frame.index.nlevels

Partial indexing works on columns too:

In [ ]:
frame["Ohio"]

### Reordering and Sorting Levels

You may need to rearrange the order of the levels on an axis. The `swaplevel` method will swap the levels. The default is to swap the levels on the rows:

In [ ]:
frame.swaplevel()

In [ ]:
frame.swaplevel(0, 1, axis=1)

You can also sort by a single level or subset of levels:

In [ ]:
frame.sort_index(level=1)

> **Student Exercise:** Swap the levels on the rows then sort the index by level `0`.

### Setting and Resetting Index

It's common to use one or more columns from a DataFrame as the row index.

In [ ]:
frame = pd.DataFrame({"a": range(7), "b": range(7, 0, -1), 
                      "c": ["one", "one", "one", "two", "two", "two", "two"], 
                      "d": [0, 1, 2, 0, 1, 2, 3]})
frame

`set_index` will return a new DataFrame using one or more of its columns as the index.

In [ ]:
frame2 = frame.set_index(["c", "d"])
frame2

`reset_index` does the opposite of `set_index` and turns the index back into a column.

In [ ]:
frame2.reset_index()

You can choose to drop the columns when resetting index:

In [ ]:
frame2.reset_index(drop=True)

## 4.2 Concatenation

You can join DataFrames along any axis which is referred to as _concatenation_ or _stacking_. This is akin to database `union` operations.

In [ ]:
s1 = pd.Series([0, 1], index=["a", "b"], dtype="Int64")
s2 = pd.Series([2, 3, 4], index=["c", "d", "e"], dtype="Int64")
s3 = pd.Series([5, 6], index=["f", "g"], dtype="Int64")

In [ ]:
pd.concat([s1, s2, s3])

By default, `concat` works along `axis="index"`, producing another Series. If you pass `axis="columns"`, the result will instead be a DataFrame:

In [ ]:
pd.concat([s1, s2, s3], axis="columns")

The default behavior of `concat` is union (`outer` join) of the indexes, you can also intersect them by passing `join='inner'`:

In [ ]:
s4 = pd.concat([s1, s3])
pd.concat([s1, s4], axis="columns", join="inner")

When combining Series along axis="columns", pass the `keys` argument for the DataFrame column headers:

In [ ]:
pd.concat([s1, s2, s3], axis="columns", keys=["one", "two", "three"])

**Student Exercise:** Concat `s1`, `s2` and `s3` along index and pass `keys=["one", "two", "three"]`.

If the index does not contain any relevant data, and you want to avoid concatenating based on indexes, you can pass the `ignore_index=True` argument:

In [ ]:
df1 = pd.DataFrame(np.random.standard_normal((3, 4)), 
                   columns=["a", "b", "c", "d"])

df2 = pd.DataFrame(np.random.standard_normal((2, 3)), 
                   columns=["b", "d", "a"])

pd.concat([df1, df2], ignore_index=True)

## 4.3 Stacking and Unstacking

These are alternative reshaping methods to Melt/Pivot that work specifically on the Index levels.

In [ ]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)), 
                    index=pd.Index(["Ohio", "Colorado"], name="state"),
                    columns=pd.Index(["one", "two", "three"], name="number"))
data

The `stack` method pivots the columns into rows, producing a Series with a MultiIndex.

In [ ]:
result = data.stack()
result

From a hierarchically indexed Series, you can rearrange the data back into a DataFrame with `unstack` , which pivots rows into columns.

In [ ]:
result.unstack()

You can unstack a different level by passing a level number or name:

In [ ]:
result.unstack(level=0)

## 4.4 Advanced GroupBy: Apply

The most general-purpose GroupBy method is `apply`, which splits the object being manipulated into pieces, invokes the passed function on each piece, and then concatenates the pieces.

Suppose we want to select the top five `tip_pct` values by group. First, write a function that selects the rows with the largest values in a particular column:

In [ ]:
def top(df, n=5, column="tip_pct"):
    return df.sort_values(column, ascending=False)[:n]

In [ ]:
top(tips, n=6)

We can then `apply` this function by different groups using `groupby`:

In [ ]:
tips.groupby("smoker").apply(top)

You can pass the arguments to the function as follows:

In [ ]:
tips.groupby(["smoker", "day"]).apply(top, n=2, column="total_bill")

**Student Exercise:** Apply the function on `day` and `time` group.

## 4.5 Advanced GroupBy: Transform

You can also transform your data using the `transform` method. It is similar to `apply` but the function must:
- Produce a scalar value to be broadcast to the shape of the group chunk, or
- Return an object that is the same shape as the group chunk

This is useful for z-score normalization within groups.

In [ ]:
df = pd.DataFrame({'key': ['a', 'b', 'c'] * 4, 'value': np.arange(12.)})
g = df.groupby('key')['value']
g.mean()

`transform` produce a Series of the same shape as `df['value']` but with values replaced by the average grouped by `key`.

In [ ]:
g.transform(lambda g: g.mean())

In [ ]:
def normalize(x):
    return (x - x.mean()) / x.std()

g.transform(normalize)